## input데이터 전처리 방안 수립
**대회에서 제공하는 원자료는 모두 삭제되었습니다**
- NH투자증권이 제공하는 금융 뉴스 데이터셋의 전처리 방안을 수립하고자 데이터의 다양한 탐색을 시도하였습니다
- 향후 BERT모델을 적용하기 위해서 KoBERT 라이브러리를 시범적으로 사용해봤습니다

<br>

## 발견한 점
- 데이터셋 내에서 특정한 데이터들의 **문장길이가 과도하게 긴 것**을 파악하였습니다
    - 12.04에 해당사항을 주최측에 문의하였는데 데이터 자체에는 문제가 없는 것으로 밝혀주셨습니다
    - 위의 내용을 견지하고 데이터셋 전처리 및 모델 학습에 유의를 기울여야 함을 알게되었습니다
- 위 문제는 train, test데이터셋 모두에서 나타났습니다. 따라서 모델 학습에 있어서 해당 사항을 처리할 방안을 고민해야 합니다
    - Padding길이와 관련된 문제가 될 것으로 추정됩니다

In [ ]:
import os, sys
from google.colab import drive

#내 구글드라이브 디렉토리 설정
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 package_collection 폴더에 패키지 저장
#매 실행마다 이 코드를 재실행해야 해당 디렉토리에 있는 패키지를 사용할 수 있음
os.symlink('/content/drive/MyDrive/Colab Notebooks/py_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


### 1. KoBERT 출력 자료 이해해보기

In [ ]:
import torch
from kobert.utils import get_tokenizer
#모델불러오는데 좀 걸림

In [ ]:
#다음의 모델을 사용함
# https://github.com/SKTBrain/KoBERT 

from kobert.pytorch_kobert import get_pytorch_kobert_model #시간 5 ~ 10분 소요

bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('/content/drive/MyDrive/dacon/news_train.csv')
train_df.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


In [ ]:
#대회 데이터를 바탕으로 문장 하나를 토큰화해보기
import re

stopwords_kr = pd.read_csv('/content/drive/MyDrive/dacon/stopwords_kr.txt',sep = '\n', header = None)
stop_kr_list = list(stopwords_kr.iloc[:,0])

url_with_https = re.compile(r'(https?:\/\/){1,}([\w.\/%-]+){1,}')
rm_email = re.compile(r'([a-zA-Z0-9\.\_]+)@([\w\.]+){1,}')
useless_words = re.compile(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]+')
rm_num = re.compile(r'(\d+\.?\,?\d+)+')

#지난 여름 자율프로젝트에서 사용했던 텍스트 전처리 방식 가져옴
def clean_kor_text(kor_text):
    text = url_with_https.sub('',kor_text)
    text = rm_email.sub('', text)
    text = useless_words.sub('',text)
    text = rm_num.sub('',text)
    text = ' '.join(word for word in text.split() if word not in stop_kr_list)
    return text

In [ ]:
train_df['clean_content'] = train_df['content'].apply(clean_kor_text)

In [ ]:
#clean_content가지고 텍스트 전처리 하기
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

sp(train_df['clean_content'][0]) #문장이 다음과 같이 토큰화 되었음, 그런데 영어가 조금 이상하게 토큰화된것으로 보임

using cached model


['▁이데일리',
 '▁M',
 'AR',
 'K',
 'E',
 'T',
 'P',
 'O',
 'IN',
 'T',
 '▁현재',
 '▁코스닥',
 '▁기관',
 '▁',
 '억',
 '▁순매도']

In [ ]:
#kobert 에서 제공하는 패키지를 사용하여 생성한 word_to_id, id_to_word
word_to_id = vocab.token_to_idx
id_to_word = vocab.idx_to_token

In [ ]:
#vcoab의 인덱스와 매칭시켜보기

test_s = sp(train_df['clean_content'][0])

test_corpus = []
for word in test_s:
  if word_to_id[word] == None:
    pass
  else:
    print(id_to_word[word_to_id[word]],':',word_to_id[word])
    test_corpus.append(word_to_id[word])
#매칭이 잘 됨

▁이데일리 : 3661
▁M : 671
AR : 267
K : 307
E : 282
T : 351
P : 333
O : 329
IN : 302
T : 351
▁현재 : 5064
▁코스닥 : 4671
▁기관 : 1261
▁ : 517
억 : 6858
▁순매도 : 2914


In [ ]:
#모델에 넣어서 pre-trianed된 분산표현을 얻기
input_test_ids = torch.LongTensor([test_corpus])

test_output = model(input_test_ids) 

In [ ]:
input_test_ids.shape

torch.Size([1, 16])

In [ ]:
test_output[0]

tensor([[[ 0.2621, -0.3518,  0.0953,  ...,  0.5056,  0.0253,  0.3087],
         [ 0.0574,  0.1657,  0.7197,  ...,  0.3358,  0.2369,  0.1387],
         [ 0.5170,  0.0986,  0.0568,  ...,  0.6123,  0.1260, -0.2050],
         ...,
         [ 0.0369, -0.1059, -0.1182,  ..., -0.0907, -0.0451, -0.2258],
         [ 0.4354, -0.1167,  0.0652,  ...,  0.2816,  0.2113,  0.1638],
         [ 0.2923,  0.0289, -0.0418,  ...,  0.3647,  0.3210, -0.0797]]],
       grad_fn=<NativeLayerNormBackward>)

In [ ]:
test_output[0].shape


torch.Size([1, 16, 768])

### 2. 데이터셋 들여다보기

In [ ]:
#모든 문장들에 대해서 sentencepiece기반 토큰화 실시

#토큰화와 매핑을 동시에 수행하는 함수 실행
def tokenize_and_mapping(sent_tokenizer, sentence, word_to_id):

  temp = []
  corpus = sent_tokenizer(sentence)

  for word in corpus:
    temp.append(word_to_id[word])

  return torch.LongTensor([temp])

In [ ]:
train_df['tokenized_cont'] = train_df['clean_content'].apply(lambda x: tokenize_and_mapping(sp, x, word_to_id))

In [ ]:
train_df.head()

,n_id,date,title,content,ord,info,clean_content,tokenized_cont
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0,이데일리 MARKETPOINT 현재 코스닥 기관 억 순매도,"[[tensor(3661), tensor(671), tensor(267), tens..."
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1,실적기반 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개,"[[tensor(3047), tensor(5561), tensor(6286), te..."
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1,하이스탁론 선취수수료 없는 최저금리 상품 출시,"[[tensor(4942), tensor(6664), tensor(7589), te..."
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0,종합 경제정보 미디어 이데일리 무단전재 재배포 금지,"[[tensor(4203), tensor(972), tensor(7229), ten..."
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0,전국적인 소비 붐 조성에 기여할 예정,"[[tensor(4014), tensor(7206), tensor(2832), te..."


In [ ]:
#문장 최대 길이는?
sent_size = train_df['tokenized_cont'].apply(lambda x: x.shape[1])

In [ ]:
sent_size.values.argmax() #가장길이가 긴 문장의 위치, 길이는 1675

37707

In [ ]:
train_df.iloc[37707, 3] #유독 이 문장만 길이가 긴것으로 나타남
  #이 문장이 속한 기사를 찾아보니 이 기사만이 문장단위로 분류가 되어 있지가 않다
  #데이터 수집 오류인가?

'지난 02월 아파트 월세 실거래가는 다음과 같다.서울지역의 아파트 월세가를 살펴보면 보증금은 최저 0만원에서 최고 17억원, 월세는 최저 1만원에서 최고 800만원인데, 강서구 등촌동 등촌1,2단지주공아파트 아파트(전용면적 26.37)가 보증금 264만원에 월세 5만원으로 최저가에, 서초구 반포동 반포자이 아파트(전용면적 194.515)가 보증금 17억원에 월세 180만원으로 최고가에 계약되었다.부산지역의 아파트 월세가는 보증금은 최저 70만원에서 최고 3억원, 월세는 최저 2만원에서 최고 200만원인데, 동래구 사직동 동일 아파트(전용면적 46.17)가 보증금 500만원에 월세 15만원으로 최저가에, 금정구 장전동 래미안장전 아파트(전용면적 84.8952)가 보증금 3억원에 월세 40만원으로 최고가에 계약되었다.대구지역의 아파트 월세가는 보증금은 최저 0만원에서 최고 9억원, 월세는 최저 5만원에서 최고 330만원인데, 달서구 월성동 월성주공3단지 아파트(전용면적 26.37)가 보증금 250만원에 월세 5만원으로 최저가에, 수성구 범어동 두산위브더제니스 아파트(전용면적 146.9701)가 보증금 9억원에 월세 330만원으로 최고가에 계약되었다.인천지역의 아파트 월세가는 보증금은 최저 100만원에서 최고 3억2000만원, 월세는 최저 6만원에서 최고 180만원인데, 중구 운서동 영종엘에이치2단지 아파트(전용면적 16.57)가 보증금 1360만원에 월세 6만원으로 최저가에, 부평구 삼산동 신성미소지움3단지 아파트(전용면적 101.48)가 보증금 3억2000만원에 월세 30만원으로 최고가에 계약되었다.광주지역의 아파트 월세가는 보증금은 최저 200만원에서 최고 3억3000만원, 월세는 최저 3만원에서 최고 100만원인데, 광산구 선암동 광주선운휴먼시아 아파트(전용면적 36.38)가 보증금 521만원에 월세 12만원으로 최저가에, 서구 화정동 현대힐스테이트3단지 아파트(전용면적 84.98)가 보증금 3억3000만원에 월세 20만원으로 최고가에 계약되었다.대전지역의

In [ ]:
sent_size.describe()
  #아무리 문장길이가 길어도 75%분위수가 29이다.

count    118745.000000
mean         23.136831
std          18.225367
min           0.000000
25%          12.000000
50%          18.000000
75%          29.000000
max        1675.000000
Name: tokenized_cont, dtype: float64

In [ ]:
#문장길이가 긴 경우가 다수 존재
sent_size.values[sent_size.values.argsort()[::-1]][:20] #상위 20개만 추출해봄

array([1675, 1289, 1080,  807,  538,  466,  450,  413,  393,  384,  358,
        351,  332,  286,  285,  285,  284,  283,  279,  276])

In [ ]:
sent_size.values.argsort()[::-1][:20]

array([37707,  3319,  2253, 42666, 16199,  6426, 20205,  2287,  9534,
       47409, 18586,  8565,  4769, 16277, 15771, 12160,  5712,    61,
        7249,  2032])

In [ ]:
train_df.iloc[sent_size.values.argsort()[::-1][:20], :]
  #이렇게 문장길이가 긴 content들은 대부분 문장단위로 구분이 된 데이터가 아니고
  #기사가 통으로 들어있는 데이터임
  #그렇다면 이를 다시 문장단위로 구분을 시켜야 하나? 그런데 info는 매핑이 불가능한데?

,n_id,date,title,content,ord,info,clean_content,tokenized_cont
37707,NEWS08612,20200304,지난달(02월) 아파트 월세 실거래가,지난 02월 아파트 월세 실거래가는 다음과 같다.서울지역의 아파트 월세가를 살펴보면...,1,0,지난 아파트 월세 실거래가는 다음과 같다서울지역의 아파트 월세가를 살펴보면 보증금은...,"[[tensor(4304), tensor(3131), tensor(517), ten..."
3319,NEWS04734,20200615,[미리보는 이데일리신문]대학 원격강의 제한 푼다…사이버대 긴장,[이데일리 최정훈 기자] 다음은 16일자 이데일리 신문 주요 뉴스다.1면-대학 원격...,1,0,이데일리 최정훈 기자 다음은 일자 이데일리 신문 주요 뉴스다1면대학 원격강의 제한 ...,"[[tensor(3661), tensor(4519), tensor(7227), te..."
2253,NEWS03462,20200112,[미리보는 이데일리신문]벤처투자 붐에도..M&A출구 꽉 막혔다,[이데일리 양희동 기자] 다음은 13일자 이데일리 신문 주요 뉴스다.1면-벤처투자 ...,1,0,이데일리 양희동 기자 다음은 일자 이데일리 신문 주요 뉴스다1면벤처투자 붐에도MA출...,"[[tensor(3661), tensor(3214), tensor(7993), te..."
42666,NEWS09295,20200311,지난 10일간 주요 부동산거래,1일에서 11일사이 각 지역별 주요 부동산 실거래가는 다음과 같다.서울지역 아파트 ...,1,0,1일에서 일사이 지역별 주요 부동산 실거래가는 다음과 같다서울지역 아파트 서초구 반...,"[[tensor(529), tensor(7126), tensor(6903), ten..."
16199,NEWS03552,20200529,"엔케이맥스, 한국 임상 1/2a상 마지막환자 투약…추적관찰 시작",엔케이맥스는 키트루다 병용 임상인 슈퍼NK(SNK01) 의 한국 임상1/2a상에서...,2,0,엔케이맥스는 키트루다 병용 임상인 슈퍼NKSNK 한국 임상a상에서 마지막 번째 환자...,"[[tensor(3292), tensor(7525), tensor(6172), te..."
6426,NEWS06180,20200214,11조원대 글로벌 시장 석권할 2월 Best종목!,오늘 종목 긴말 필요없다. 오늘 장 마감 직전이라도 상한가로 말아올릴만큼 매물의 씨...,1,1,오늘 종목 긴말 필요없다 오늘 장 마감 직전이라도 상한가로 말아올릴만큼 매물의 씨가...,"[[tensor(3419), tensor(4202), tensor(1311), te..."
20205,NEWS04588,20200623,"현대차그룹, 로드아일랜드 디자인스쿨과 미래 모빌리티 디자인 혁신 위한 협업",자연 생명체와 생태계에 대한 다양하고 심도 깊은 연구활동을 지원하고 있으며 RISD...,4,0,자연 생명체와 생태계에 대한 다양하고 심도 깊은 연구활동을 지원하고 있으며 RISD...,"[[tensor(3916), tensor(2712), tensor(7436), te..."
2287,NEWS01541,20200627,과학기술정보통신부 주간계획(6월 29~7월 3일),[이데일리 이후섭 기자] 다음주(6월 29~7월 3일) 과학기술정보통신부 주요일정 ...,2,0,이데일리 이후섭 기자 다음주6월 3일 과학기술정보통신부 주요일정 보도계획이다주간 주...,"[[tensor(3661), tensor(3756), tensor(6570), te..."
9534,NEWS03256,20200611,KOSPI 200 Closing Price List-4,KOSPI 200 Closing Price List-4KOSPI 200 Closin...,1,0,KOSPI Closing Price List4KOSPI Closing Price L...,"[[tensor(659), tensor(331), tensor(337), tenso..."
47409,NEWS01166,20200325,"전국 소상공인 단체 ""이동주 더불어시민당 비례대표 후보 환영""","(소속단체) 한국지역경제살리기중앙회, 한국소상공인정책포럼, 전국수퍼마켓협동조합연합회...",16,0,소속단체 한국지역경제살리기중앙회 한국소상공인정책포럼 전국수퍼마켓협동조합연합회 한국발...,"[[tensor(2837), tensor(5791), tensor(4958), te..."


In [ ]:
#test 데이터의 경우에는? 어떤가?

test_df = pd.read_csv('/content/drive/MyDrive/dacon/news_test.csv')
test_df.head()

,n_id,date,title,content,ord,id
0,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1,NEWS00237_1
1,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2,NEWS00237_2
2,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3,NEWS00237_3
3,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4,NEWS00237_4
4,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대","코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5,NEWS00237_5


In [ ]:
#내용 전처리
test_df['clean_content'] = test_df['content'].apply(clean_kor_text)

#토큰화 및 매핑
test_df['tokenized_cont'] = test_df['clean_content'].apply(lambda x: tokenize_and_mapping(sp, x, word_to_id))

In [ ]:
test_df.head()

,n_id,date,title,content,ord,id,clean_content,tokenized_cont
0,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1,NEWS00237_1,마이크로 LED TV 장비 양산 돌입 전방업체 투자 확대로 본업도 호조연일 신고가,"[[tensor(1907), tensor(7096), tensor(7565), te..."
1,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2,NEWS00237_2,이데일리 김대웅 기자 반도체 장비 업체 코세스의 기술력이 마이크로 발광다이오드LED...,"[[tensor(3661), tensor(1316), tensor(5808), te..."
2,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3,NEWS00237_3,최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,"[[tensor(4525), tensor(1690), tensor(3269), te..."
3,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4,NEWS00237_4,TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,"[[tensor(694), tensor(6116), tensor(4916), ten..."
4,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대","코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5,NEWS00237_5,코세스는 반도체 장비를 제조 판매하는 업체로 지난 코스닥 시장에 상장했고 창업주인 ...,"[[tensor(4665), tensor(6579), tensor(6666), te..."


In [ ]:
test_sent_size = test_df['tokenized_cont'].apply(lambda x: x.shape[1])

In [ ]:
test_sent_size.describe()
  #테스트 문서 내에도 문장단위로 처리되지 않은 데이터가 있음

count    142565.00000
mean         23.29764
std          17.28816
min           0.00000
25%          12.00000
50%          19.00000
75%          30.00000
max        1575.00000
Name: tokenized_cont, dtype: float64

In [ ]:
test_sent_size.values[test_sent_size.values.argsort()[::-1]][:50] #상위 50개만 추출해봄

array([1575,  771,  572,  542,  441,  382,  340,  312,  311,  307,  283,
        283,  281,  275,  275,  275,  275,  272,  271,  261,  260,  256,
        254,  253,  251,  251,  239,  239,  228,  225,  223,  223,  218,
        214,  213,  211,  210,  209,  209,  208,  208,  203,  200,  197,
        195,  194,  192,  192,  191,  190])